In [2]:
from selenium.webdriver import Firefox, Chrome
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.firefox.options import Options
import time

In [3]:
opts = Options()
opts.add_argument("--headless")
#assert opts.headless
#browser = Firefox(executable_path='geckodriver', options=opts)
browser = Chrome(executable_path='chromedriver', options=opts)
url = 'https://freida.ama-assn.org/institution/120127'
browser.get(url)
time.sleep(10)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 107
Current browser version is 106.0.5249.103 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x006E8CE3+2067683]
	Ordinal0 [0x0067CBC1+1625025]
	Ordinal0 [0x0057B7CD+571341]
	Ordinal0 [0x0059DFED+712685]
	Ordinal0 [0x00599286+692870]
	Ordinal0 [0x00596BF9+683001]
	Ordinal0 [0x005CDBF6+908278]
	Ordinal0 [0x005CD6CA+906954]
	Ordinal0 [0x005C7156+880982]
	Ordinal0 [0x0059FF1D+720669]
	Ordinal0 [0x005A0F5F+724831]
	GetHandleVerifier [0x0099F162+2771330]
	GetHandleVerifier [0x00991145+2713957]
	GetHandleVerifier [0x0077CC0A+533546]
	GetHandleVerifier [0x0077B970+528784]
	Ordinal0 [0x00684F4C+1658700]
	Ordinal0 [0x006898B8+1677496]
	Ordinal0 [0x006899A5+1677733]
	Ordinal0 [0x00694CC4+1723588]
	BaseThreadInitThunk [0x767BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AE7B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77A